In [1]:
import math
import itertools
import numpy as np

import hoomd
import gsd.hoomd
import coxeter

#### particle vertices, faces, and insphere radius

In [2]:
lattice = gsd.hoomd.open('./DATA/lattice.gsd')
verts = lattice[0].particles.type_shapes[0]['vertices']
particle = coxeter.shapes.ConvexPolyhedron(verts)
faces = particle.faces
sigma = 2*particle.insphere_from_center.radius

sphere = coxeter.shapes.Ellipsoid(0.5, 0.5, 0.5)
sigma2 = sphere.maximal_bounded_sphere_radius * 2

#### parameters

In [3]:
# integration timestep
dt=0.0005

# coupling
kT=1.0
tau=100*dt

# reduced pressure
pressure=16*kT/particle.volume
tauS=1000*dt
pressure

2.0

#### simulation

In [4]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=20)
sim.create_state_from_gsd(filename='./DATA/compressed.gsd', frame=-1)

#### potential

In [5]:
# neighboring list
nl = hoomd.md.nlist.Cell(buffer=0.4)

# potential
r_cut_A = 2*particle.circumsphere_from_center.radius + 0.15*sigma + 0.167*sigma*2
r_cut_B = sigma2 + 0.15*sigma2
beta=0.15
lambda_min=2**(1/6)
lambda_attr=2.5
r_cut_AB = max(lambda_min*sigma, r_cut_A + r_cut_B + 0.5*lambda_attr*beta*(sigma+sigma2))

alj = hoomd.md.pair.aniso.ALJ(nl)
alj.r_cut[('A', 'A')] = r_cut_A
alj.params[('A', 'A')] = dict(epsilon=0.1, 
                              sigma_i=sigma, 
                              sigma_j=sigma, 
                              alpha=0)

alj.r_cut[('B', 'B')] = r_cut_B
alj.params[('B', 'B')] = dict(epsilon=0.1, 
                              sigma_i=sigma2, 
                              sigma_j=sigma2, 
                              alpha=0)

alj.r_cut[('A', 'B')] = r_cut_AB
alj.params[('A', 'B')] = dict(epsilon=0.1, 
                              sigma_i=sigma, 
                              sigma_j=sigma2, 
                              alpha=2)

alj.shape['A'] = dict(vertices=verts, 
                      faces=faces, 
                      rounding_radii=0.167*sigma) # rounding_radii=0.15*(sigma/2)
alj.shape['B'] = dict(vertices = [], 
                      faces = [], 
                      rounding_radii = (sphere.a, sphere.b, sphere.c))


#### thermodynamic ensemble

In [6]:
npt = hoomd.md.methods.NPT(filter=hoomd.filter.All(), 
                           kT=kT, 
                           tau=tau, 
                           S=pressure, 
                           tauS=tauS,
                           couple='xyz')

#### integrator

In [7]:
integrator = hoomd.md.Integrator(dt=dt, 
                                 methods=[npt],
                                 forces=[alj],
                                 integrate_rotational_dof=True)

#### writer

In [8]:
thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())

In [9]:
logger = hoomd.logging.Logger()
logger.add(sim, quantities=['timestep', 'walltime'])
logger.add(alj, quantities=['type_shapes'])
logger.add(thermodynamic_properties)

gsd_writer = hoomd.write.GSD(filename='./DATA/trajectory.gsd',
                             trigger=hoomd.trigger.Periodic(100),
                             log=logger,
                             mode='wb')

In [10]:
sim.operations.writers.append(gsd_writer)
sim.operations.integrator = integrator
sim.operations.computes.append(thermodynamic_properties)

#### run

In [11]:
sim.run(0)
# npt.thermalize_thermostat_and_barostat_dof()
alj.type_shapes

[{'type': 'ConvexPolyhedron',
  'rounding_radius': 0.484,
  'vertices': [[1, 1, 1],
   [1, 1, -1],
   [1, -1, 1],
   [-1, 1, 1],
   [-1, 1, -1],
   [1, -1, -1],
   [-1, -1, 1],
   [-1, -1, -1]]},
 {'type': 'Ellipsoid', 'a': 0.575, 'b': 0.575, 'c': 0.575}]

In [12]:
sim.run(1e+6)

RuntimeError: nlist: Simulation box is too small! Particles would be interacting with themselves.rmax=6.48721
nearest_plane_distance.x=12.9672
nearest_plane_distance.y=12.9672
nearest_plane_distance.z=12.9672


In [ ]:
phi = particle.volume*sim.state.N_particles/sim.state.box.volume
print(phi)